<a href="https://colab.research.google.com/github/KOCORA/Applied-Artificial-Intelligence/blob/master/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [ ]:
#!pip install pymorphy2
#import pymorphy2 as pm

In [11]:
# Télécharger le fichier

data = pd.read_excel('/content/drive/MyDrive/Templates/full word_rating_after_coding.xlsx')
data.head

<bound method NDFrame.head of          абажур  0
0        абажур  0
1        абажур -1
2      абориген -1
3      абориген -1
4      абориген  0
...         ... ..
26765      яшин  0
26766      ящик  0
26767      ящик  0
26768      ящик  0
26769      ящик  0

[26770 rows x 2 columns]>

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26770 entries, 0 to 26769
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   абажур  26770 non-null  object
 1   0       26770 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 418.4+ KB


In [17]:
data_positive = pd.read_csv('/content/drive/MyDrive/Templates/positive.csv', sep=';', header=None)
data_positive['remark'] = 1

data_negative = pd.read_csv('/content/drive/MyDrive/Templates/negative.csv', sep=';', header=None)
data_negative['remark'] = -1
df = pd.concat([data_positive, data_negative]).loc[:, [3, "remark"]]
df.rename(columns={3: 'text'}, inplace=True)

df

,text,remark
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1
...,...,...
111918,Но не каждый хочет что то исправлять:( http://...,-1
111919,скучаю так :-( только @taaannyaaa вправляет мо...,-1
111920,"Вот и в школу, в говно это идти уже надо(",-1
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1


In [18]:
df.info

<bound method DataFrame.info of                                                      text  remark
0       @first_timee хоть я и школота, но поверь, у на...       1
1       Да, все-таки он немного похож на него. Но мой ...       1
2       RT @KatiaCheh: Ну ты идиотка) я испугалась за ...       1
3       RT @digger2912: "Кто то в углу сидит и погибае...       1
4       @irina_dyshkant Вот что значит страшилка :D\nН...       1
...                                                   ...     ...
111918  Но не каждый хочет что то исправлять:( http://...      -1
111919  скучаю так :-( только @taaannyaaa вправляет мо...      -1
111920          Вот и в школу, в говно это идти уже надо(      -1
111921  RT @_Them__: @LisaBeroud Тауриэль, не грусти :...      -1
111922  Такси везет меня на работу. Раздумываю приплат...      -1

[226834 rows x 2 columns]>

In [19]:
df.text = df.text.apply(lambda x: str.strip(re.sub(r"[^а-яА-Я\s]+", "", x)))
df = df.sample(frac=1.0)
df.reset_index(drop=True, inplace=True)
df = df.head(20050)
df

,text,remark
0,Нашел на что убивать внутренний траф слушаю д...,1
1,да я с на работе еще хорошо утром когда не с...,-1
2,забей ты на всех ты офигенная все нормально будет,1
3,И мне и мне такое,1
4,еба а че меня не позвали Небось там сиги курит...,-1
...,...,...
20045,Заболела блин валяюсь дома с температурой подыхаю,-1
20046,Проводила Никитку\nВот теперь думаю спать не с...,1
20047,а вообще я сперва тебе раскадровку должна,1
20048,Кстати Хоббит впечатлил но фильм одноразовый н...,-1


In [20]:
!pip install pymorphy2
import pymorphy2 as pm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 51.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=55bc36d5be34e647608c284bf5974400338f297c17fedfdbd738193b43b44506
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [21]:
morph = pm.MorphAnalyzer()
nltk.download('punkt')

for index, row in df.iterrows():
    tokens = nltk.word_tokenize(row.text)
    lemmatized = []
    pos_tags = []
    
    for token in tokens:
        lemmatized.append(morph.parse(token)[0].normal_form)
        pos_tags.append(morph.parse(token)[0].tag.POS)
    
    text_data = []
    for word in lemmatized:
        if word in data.index.values:
            text_data.append(data.loc[word])
            
    if len(text_data) == 0:
        text_data.append(0)

    df.at[index, 'X1'] = np.mean(text_data)
    df.at[index, 'X2'] = np.max(text_data)
    df.at[index, 'X3'] = np.min(text_data)
    df.at[index, 'X4'] = np.sum(text_data)
    df.at[index, 'X5'] = np.sum(np.array(text_data) > 0)
    df.at[index, 'X6'] = np.sum(np.array(text_data) < 0)

    pos_counts = pd.Series(pos_tags).value_counts()
    pos_names = ['NOUN', 'ADJF', 'ADJS', 'VERB', 'INFN', 'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ']
    message_len = len(row.text.split())

    for i, pos in enumerate(pos_names):
        value = 0
        if pos in pos_counts.index.values:
            value = pos_counts[pos]
        value /= message_len
        df.at[index, f'X{7+i}'] = value

df

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
<ipython-input-21-c146ca12b268>:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if word in data.index.values:
<ipython-input-21-c146ca12b268>:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if word in data.index.values:
<ipython-input-21-c146ca12b268>:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if word in data.index.values:
<ipython-input-21-c146ca12b268>:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if word in data.index.values:
<ipython-input-21-c146ca12b268>:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform element

,text,remark,X1,X2,X3,X4,X5,X6,X7,X8,...,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20
0,Нашел на что убивать внутренний траф слушаю д...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.111111,...,0.111111,0.0,0.0000,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000
1,да я с на работе еще хорошо утром когда не с...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.263158,0.000000,...,0.000000,0.0,0.0000,0.157895,0.052632,0.052632,0.157895,0.105263,0.157895,0.000000
2,забей ты на всех ты офигенная все нормально будет,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.222222,...,0.000000,0.0,0.0000,0.111111,0.222222,0.000000,0.111111,0.000000,0.111111,0.000000
3,И мне и мне такое,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.200000,...,0.000000,0.0,0.0000,0.000000,0.400000,0.000000,0.000000,0.400000,0.000000,0.000000
4,еба а че меня не позвали Небось там сиги курит...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.222222,0.000000,...,0.000000,0.0,0.0000,0.055556,0.166667,0.000000,0.000000,0.111111,0.166667,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20045,Заболела блин валяюсь дома с температурой подыхаю,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,0.000000,...,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.142857
20046,Проводила Никитку\nВот теперь думаю спать не с...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.250000,0.0,0.0000,0.125000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000
20047,а вообще я сперва тебе раскадровку должна,1,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.000000,...,0.000000,0.0,0.0000,0.285714,0.285714,0.000000,0.000000,0.142857,0.000000,0.000000
20048,Кстати Хоббит впечатлил но фильм одноразовый н...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.375000,0.125000,...,0.062500,0.0,0.0625,0.000000,0.000000,0.062500,0.125000,0.125000,0.000000,0.000000


In [22]:
#Entraînement des données

X = df.iloc[:, 2:]
y = df.iloc[:, 1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

model = LogisticRegression().fit(X_train, y_train)

y_pred = model.predict(X_test)

In [23]:
#Evaluation du modèle avec la metric F1

f1 = f1_score(y_test, y_pred)
f1

0.5838599902771027

In [28]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1097,  840],
       [ 872, 1201]])

In [25]:
#Resultat du data final

data_final = {'predicted': y_pred, 'true': y_test}

final_df = pd.DataFrame(data_final)
final_df

,predicted,true
8000,1,-1
18761,1,-1
19384,1,1
8020,1,1
14251,-1,-1
...,...,...
19946,1,-1
15062,1,-1
691,-1,1
12269,1,-1


In [27]:
# Enregistrement du fichiers "sentiment.csv"

final_df.to_csv('/content/drive/MyDrive/Templates/sentiment.csv')

sentiment = pd.read_csv('/content/drive/MyDrive/Templates/sentiment.csv')
sentiment

,Unnamed: 0,predicted,true
0,8000,1,-1
1,18761,1,-1
2,19384,1,1
3,8020,1,1
4,14251,-1,-1
...,...,...,...
4005,19946,1,-1
4006,15062,1,-1
4007,691,-1,1
4008,12269,1,-1
